In [6]:
import torch 
import lightning as ln
import torch.nn as nn 
import torch.nn.functional as Func 
import pytorch_lightning as pl 
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader

import os
import pandas as pd
from pandas import DataFrame
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [7]:
train_path = 'cyberbullying_dataset.csv'
dataset = pd.read_csv(train_path)

In [8]:
print(dataset['cyberbullying_type'].value_counts())
print('#' * 20)
print(dataset.info())

cyberbullying_type
5    7998
0    7992
2    7973
1    7961
3    7945
4    7823
Name: count, dtype: int64
####################
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47692 entries, 0 to 47691
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_text          47692 non-null  object
 1   cyberbullying_type  47692 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 745.3+ KB
None


In [9]:
le = LabelEncoder()

dataset['cyberbullying_type'] = le.fit_transform(dataset['cyberbullying_type'])
classes  = le.classes_

print(classes)

[0 1 2 3 4 5]


In [10]:
tokenizer = get_tokenizer('basic_english')



<function torchtext.data.utils._basic_english_normalize(line)>

In [11]:
# Accessing the data

class TweetDataset(Dataset):
    def __init__(self, csv_file, batch_size=32):
        super().__init__()
        self.csv_file = csv_file
        self.batch_size = batch_size
        self.tokenizer = get_tokenizer('basic_english')
        self.vocab = None

    def setup(self, stage=None):
        data = pd.read_csv(self.csv_file)
        
        self.train_data, self.val_data = train_test_split(data, test_size=0.2, random_state=42)
        self.test_data = self.val_data

        text_iterator = (text for text in self.train_data['tweet_text'])
        self.vocab = build_vocab_from_iterator(text_iterator, specials=['<UNK>'])
        self.vocab.set_default_index(self.vocab['<UNK>'])
        
        self.input_size = len(elf.vocab)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()
        
        sample = self.data.iloc[index]

        if self.transform:
            sample = self.transform(sample)

        sample_dict = sample.to_dict()

        if self.transform:
            sample_dict = self.transform(sample_dict)

        return sample_dict

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True, collate_fn=self.collate_fn)
    
    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size, shuffle=False, collate_fn=self.collate_fn)

    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size=self.batch_size, shuffle=False, collate_fn=self.collate_fn)

    def collate_fn(self, batch):
        text = [self.vocab(self.tokenizer(row['tweet_text'])) for row in batch]
        label = [row['cyberbullying_type'] for row in batch]

        return torch.tensor(text), torch.tensor(label)


In [13]:
# Imports for Pytorch Lightning data loader

train_text_data, val_text_data = split_data('cyberbullying_dataset.csv')
train_dataloader = DataLoader(train_text_data, batch_size=32, shuffle=True, num_workers=4)

In [ ]:
# class LSTMClassifier(nn.Module):
#     def __init__(
#         self,
#         embedddings: torch.Tensor,
#         lstm_hidden_size: int = 128,
#         lstm_num_layers: int = 1
#     )

